In [1]:
!pip install --no-cache-dir --upgrade git+https://github.com/evfro/polara.git@develop#egg=polara

  Cloning https://github.com/evfro/polara.git (to revision develop) to /tmp/pip-install-gvttmx1n/polara_882dc616f41f41fd96f834d36ec7c514
  Running command git clone --filter=blob:none -q https://github.com/evfro/polara.git /tmp/pip-install-gvttmx1n/polara_882dc616f41f41fd96f834d36ec7c514
  Running command git checkout -b develop --track origin/develop
  Switched to a new branch 'develop'
  Branch 'develop' set up to track remote branch 'develop' from 'origin'.
  Resolved https://github.com/evfro/polara.git to commit ef7a5360e3f9793f4f0fdd4f31e32399d2e52069
  Preparing metadata (setup.py) ... done
  Created wheel for polara: filename=polara-0.7.2.dev0-py3-none-any.whl size=89444 sha256=841aa40983b1d68bf2bfba6bab6d582e071529b6229a4d24e49d21e19234724a
  Stored in directory: /tmp/pip-ephem-wheel-cache-ek2qng66/wheels/19/88/81/920c4189a6b0b92f19b02f18fcb44ded22ae852f7b09ab2e28
Successfully built polara


In [8]:
!pip3 install numba

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [3]:
!pip install clearml

  Using cached clearml-1.16.5-py2.py3-none-any.whl (1.2 MB)
  Using cached furl-2.1.3-py2.py3-none-any.whl (20 kB)
  Using cached pathlib2-2.3.7.post1-py2.py3-none-any.whl (18 kB)
  Using cached PyJWT-2.8.0-py3-none-any.whl (22 kB)
  Using cached orderedmultidict-1.0.1-py2.py3-none-any.whl (11 kB)
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 2.3.0
    Uninstalling PyJWT-2.3.0:
      Successfully uninstalled PyJWT-2.3.0


In [4]:
 !pip install hydra-core 

  Using cached hydra_core-1.3.2-py3-none-any.whl (154 kB)
  Using cached omegaconf-2.3.0-py3-none-any.whl (79 kB)
  Using cached antlr4_python3_runtime-4.9.3-py3-none-any.whl


In [5]:
!pip install omegaconf

In [9]:
!tar -xvzf ../does-it-split.tar.gz

does-it-split/
does-it-split/data/
does-it-split/data/Movielens.csv
does-it-split/data/.ipynb_checkpoints/
does-it-split/runs/
does-it-split/runs/conf/
does-it-split/runs/conf/rules.yaml
does-it-split/runs/conf/prepr/
does-it-split/runs/conf/prepr/prepr.yaml
does-it-split/runs/conf/prepr/.ipynb_checkpoints/
does-it-split/runs/conf/prepr/.ipynb_checkpoints/prepr-checkpoint.yaml
does-it-split/runs/conf/statistic_test.yaml
does-it-split/runs/conf/datasets_info/
does-it-split/runs/conf/datasets_info/Movielens-20.yaml
does-it-split/runs/conf/datasets_info/Yelp.yaml
does-it-split/runs/conf/datasets_info/Games.yaml
does-it-split/runs/conf/datasets_info/Steam.yaml
does-it-split/runs/conf/datasets_info/Diginetica.yaml
does-it-split/runs/conf/datasets_info/Gowalla.yaml
does-it-split/runs/conf/datasets_info/Zvuk.yaml
does-it-split/runs/conf/datasets_info/Otto.yaml
does-it-split/runs/conf/datasets_info/Movielens.yaml
does-it-split/runs/conf/datasets_info/30Music.yaml
does-it-split/runs/conf/datase

In [5]:
%load_ext autoreload
%autoreload 2
    
from data import get_dataset, data_to_sequences, data_to_ratequences, data_to_ratequences_splitted, data_to_ratequences_splitted_hist
from eval_utils import sasrec_model_scoring

from time import time
from functools import reduce
import uuid
from tqdm import tqdm
import os

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader

from model import SASRec
from data import get_dataset, data_to_sequences, SequentialDataset
from utils import topn_recommendations, downvote_seen_items
from eval_utils import model_evaluate, sasrec_model_scoring, get_test_scores
from new_split.preprocessing.preprocessing import preprocessing

#from sklearn.linear_model import LogisticRegression
from train import build_sasrec_model

from typing import Dict
from typing import Optional
from typing import Union

from sklearn.utils import check_random_state

import matplotlib.pyplot as plt
import pickle
import pyarrow.parquet as pq
import datetime

#from metrics import Evaluator
from polara import get_movielens_data 
from polara.preprocessing.dataframes import reindex, leave_one_out

from rl_ope.utils import prepare_svd

# os.environ['CUDA_VISIBLE_DEVICES'] = '2'
device='cuda:1'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
torch.cuda.is_available()

True

In [8]:
import torch.nn as nn

model = nn.Sequential(
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 240000)
        )

for name, layer in model.named_modules():
    if isinstance(layer, nn.Linear):
        print(name)
        weight_norm = torch.norm(layer.weight, p=2)
        print(f"Layer {name}: L2 Norm of weights = {weight_norm.item():.4f}")

Sequential(
  (0): Linear(in_features=128, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=512, bias=True)
  (3): ReLU()
  (4): Linear(in_features=512, out_features=240000, bias=True)
)
<generator object Module.modules at 0x7feee8becdd0>
0
Layer 0: L2 Norm of weights = 13.0551
2
Layer 2: L2 Norm of weights = 13.0479
4
Layer 4: L2 Norm of weights = 282.7120


In [3]:
import random
all_actions = np.arange(250000)
state_seq = np.arange(100)
allowed_actions = all_actions[~np.isin(all_actions, state_seq)]

In [4]:
%%timeit

actions_neg = random.sample(range(250000), k=100)
# actions_neg = np.random.randint(250000, size=(8, 100))
# actions_neg = np.random.choice(all_actions, 100, replace=False)

65 µs ± 448 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [10]:
a = torch.randn(32, 128)

a.unsqueeze(1).expand(-1, 10, 128).shape

torch.Size([32, 10, 128])

## Data

In [3]:
# должны быть колонки userid, itemid, timestamp
#dataset = 'None' # сначала на мл1 попробуй запустить
dataset = pd.read_parquet('../zvuk_datasets/zvuk_1day_clickstream_210924_with_timestamp.parquet', engine='auto')
# dataset = pq.read_table('../zvuk_datasets/zvuk_clickstream_40days_with_timestamp.parquet',
                         # columns=['user_id', 'track_id', 'timestamp', 'playevent_play_duration', 'day'])

In [5]:
dataset.columns

Index(['src_id', 'day', 'user_id', 'timestamp', 'track_id', 'app',
       'playevent_play_duration', 'src_type', 'subscription_type', 'editor_id',
       'event_from_recommendation', 'group'],
      dtype='object')

In [ ]:
dataset = dataset.to_pandas()

In [ ]:
dataset.shape

In [ ]:
dataset = dataset[dataset['timestamp'] >= 1e15]

In [ ]:
dataset.shape

In [8]:
dataset

,user_id,track_id,timestamp,playevent_play_duration,day
0,479071,115025877.0,1.727867e+15,1,2024-10-02
1,479071,138206302.0,1.727867e+15,1,2024-10-02
2,479071,129898539.0,1.727867e+15,1,2024-10-02
3,479071,58553470.0,1.727867e+15,1,2024-10-02
4,479071,120470213.0,1.727868e+15,1,2024-10-02
...,...,...,...,...,...
725410289,1187140,137159381.0,1.728091e+15,3,2024-10-05
725410290,1187140,137159381.0,1.728091e+15,3,2024-10-05
725410291,1187140,57910930.0,1.728091e+15,79,2024-10-05
725410292,1187140,91700885.0,1.728091e+15,5,2024-10-05


In [9]:
test_timepoint = dataset['timestamp'].quantile(
            q=0.6, interpolation='nearest'
)


dataset = dataset.query(
    'timestamp < @test_timepoint')

In [10]:
dataset

,user_id,track_id,timestamp,playevent_play_duration,day
0,479071,115025877.0,1.727867e+15,1,2024-10-02
1,479071,138206302.0,1.727867e+15,1,2024-10-02
2,479071,129898539.0,1.727867e+15,1,2024-10-02
3,479071,58553470.0,1.727867e+15,1,2024-10-02
4,479071,120470213.0,1.727868e+15,1,2024-10-02
...,...,...,...,...,...
725410289,1187140,137159381.0,1.728091e+15,3,2024-10-05
725410290,1187140,137159381.0,1.728091e+15,3,2024-10-05
725410291,1187140,57910930.0,1.728091e+15,79,2024-10-05
725410292,1187140,91700885.0,1.728091e+15,5,2024-10-05


In [11]:
dataset = dataset.rename(columns={'user_id': 'userid', 'track_id': 'itemid'})

In [12]:
np.unique(dataset['userid'].values).shape

(1815075,)

In [13]:
np.unique(dataset['itemid'].values).shape

(3875529,)

In [14]:
#filtered_data = preprocessing(dataset, item_min_count=100, min_len=10, min_duration=30, if_drop_repeats=True)
filtered_data = preprocessing(dataset, users_sample=None, item_min_count=101, min_len=10, core=False, encoding=False,
                  drop_repeats=False, user_id='userid', item_id='itemid', timestamp='timestamp',
                  path_to_save=None)

raw data: interactions 435245672 
after item filtering: interactions 275677475 
after dropping short sequences: interactions 274138745 
after item filtering: interactions 253574407 
after dropping short sequences: interactions 253573539 


In [15]:
# filtered_data = filtered_data.drop(columns=['playevent_play_duration', 'day'])

In [16]:
filtered_data.shape

(253573539, 5)

In [17]:
filtered_data = filtered_data.rename(columns={'user_id': 'userid', 'item_id': 'itemid'})

In [18]:
filtered_data

,userid,itemid,timestamp,playevent_play_duration,day
7,479071,137920044.0,1.727867e+15,112,2024-10-02
8,479071,112091838.0,1.727867e+15,154,2024-10-02
10,479071,138163265.0,1.727867e+15,54,2024-10-02
12,479071,138262387.0,1.727867e+15,217,2024-10-02
28,1094149,50584863.0,1.729807e+15,55,2024-10-24
...,...,...,...,...,...
725410235,1375603,123344514.0,1.727979e+15,176,2024-10-03
725410236,1375603,69586325.0,1.727978e+15,238,2024-10-03
725410237,1375603,139661482.0,1.727978e+15,156,2024-10-03
725410285,1187140,135509762.0,1.728091e+15,80,2024-10-05


In [19]:
np.unique(filtered_data['userid'].values).shape

(1148455,)

In [20]:
np.unique(filtered_data['itemid'].values).shape

(232615,)

In [21]:
training_temp, data_description_temp, testset_valid_temp, testset, holdout_valid_temp, _ = get_dataset(path=filtered_data, splitting='temporal_full',
                                                                                     q=0.8)
data_description_temp

Filtered 159165 invalid observations.
Filtered 5263 invalid observations.


{'users': 'userid',
 'items': 'itemid',
 'order': 'timestamp',
 'n_users': 1078263,
 'n_items': 232274}

In [22]:
cnt_users_val = 1024
users_in_val = np.unique(testset_valid_temp['userid'].values)
ind_val = np.random.choice(users_in_val.shape[0], cnt_users_val, replace=False)
users_in_val = users_in_val[ind_val]

In [23]:
testset_valid_temp_cut = testset_valid_temp.copy()
testset_valid_temp_cut = testset_valid_temp_cut.loc[testset_valid_temp_cut['userid'].isin(users_in_val)]

In [24]:
holdout_valid_temp_cut = holdout_valid_temp.copy()
holdout_valid_temp_cut = holdout_valid_temp_cut.loc[holdout_valid_temp_cut['userid'].isin(users_in_val)]

In [25]:
testset_valid_temp_cut.sort_values(by='userid', inplace=True)

In [26]:
holdout_valid_temp_cut.sort_values(by='userid', inplace=True)

In [27]:
testset_valid_temp_cut

,userid,itemid,timestamp,playevent_play_duration,day
84242469,332,218799,1.728074e+15,40,2024-10-04
84242534,332,218799,1.728076e+15,40,2024-10-04
79336404,332,42642,1.728075e+15,40,2024-10-04
593708360,332,93577,1.728074e+15,40,2024-10-04
593405890,332,78019,1.728074e+15,40,2024-10-04
...,...,...,...,...,...
62759790,1078241,38954,1.728340e+15,339,2024-10-07
461093579,1078241,22955,1.728265e+15,72,2024-10-07
62758845,1078241,73114,1.729024e+15,59,2024-10-15
62755051,1078241,38954,1.728174e+15,329,2024-10-06


In [28]:
holdout_valid_temp_cut

,userid,itemid,timestamp,playevent_play_duration,day
520992642,332,126547,1.728152e+15,40,2024-10-05
90183055,615,227474,1.729378e+15,31,2024-10-19
322649645,1184,0,1.729468e+15,1139,2024-10-20
314758364,2259,20955,1.729168e+15,85,2024-10-17
313382086,3389,27480,1.727984e+15,83,2024-10-03
...,...,...,...,...,...
349788043,1075491,50868,1.728741e+15,147,2024-10-12
624292666,1076798,110248,1.729456e+15,191,2024-10-20
578888017,1077741,230446,1.728738e+15,236,2024-10-12
334163905,1077813,231721,1.729362e+15,144,2024-10-21


In [29]:
training_temp.to_csv('training_temp.csv', index=False) 
testset_valid_temp_cut.to_csv('testset_valid_temp_cut.csv', index=False) 
holdout_valid_temp_cut.to_csv('holdout_valid_temp_cut.csv', index=False) 
testset.to_csv('testset.csv', index=False)

In [30]:
data_description_temp

{'users': 'userid',
 'items': 'itemid',
 'order': 'timestamp',
 'n_users': 1078263,
 'n_items': 232274}

In [7]:
training_temp = pd.read_csv('training_temp.csv')
testset_valid_temp_cut = pd.read_csv('testset_valid_temp_cut.csv')
holdout_valid_temp_cut = pd.read_csv('holdout_valid_temp_cut.csv')
data_description_temp = {'users': 'userid',
 'items': 'itemid',
 'order': 'timestamp',
 'n_users': training_temp.userid.nunique(),
 'n_items': training_temp.itemid.max()}

training_temp['rating'] = np.ones(training_temp.shape[0])

In [8]:
training_temp["itemid"].max()

232273

In [8]:
item_embs = prepare_svd(training_temp, data_description_temp, rank=128, device=device)

KeyboardInterrupt: 

In [ ]:
torch.save(item_embs, "./saved_models/item_embs.pt")

## Train SASRec models

In [4]:
base_config1 = dict(
    manual_seed = 123,
    sampler_seed = 123,
    num_epochs = 15,
    maxlen = 100,
    hidden_units = 128, #
    dropout_rate = 0.3,#0.3,
    num_blocks = 2,
    num_heads = 2,
    batch_size = 64,
    learning_rate = 1e-3,
    fwd_type = 'ce',
    l2_emb = 0,
    patience = 10,
    skip_epochs = 1,
    n_neg_samples=0,
    sampling='no_sampling'
)

In [5]:
# item_embs = torch.load("./saved_models/item_embs.pt", map_location=torch.device(device))
item_embs = None

In [6]:
# model_bce = build_sasrec_model(base_config_bce, train, data_description_zvuk, testset_valid, holdout_valid, device, task=None, log=None)

In [7]:
base_config_bce = dict(
    manual_seed = 123,
    sampler_seed = 123,
    init_emb_svd = None,
    lin_layer_dim = -1,
    num_epochs = 5, #3 10 22 100&dropout0.9&hd32&bs1000
    maxlen = 100,
    hidden_units = 128,
    dropout_rate = 0.3,
    num_blocks = 2,
    num_heads = 2,
    batch_size = 128,
    learning_rate = 1e-3,
    fwd_type = 'bce',
    l2_emb = 0,
    patience = 10,
    skip_epochs = 1,
    n_neg_samples=600,
    sampling='without_rep'
)

model_e = build_sasrec_model(base_config_bce,
                             training_temp,
                             data_description_temp,
                             testset_valid_temp_cut,
                             holdout_valid_temp_cut,
                             device,
                             task=None,
                             log=None,
                             item_emb_svd=item_embs)

  0%|          | 0/5 [00:00<?, ?it/s]

loss=0.006480527881862786, epoch=0
Epoch 0, NDCG@10: 0.18982870055738182
Epoch 0, HR@10: 0.3115234375
Epoch 0, Cov@10: 0.027562394251591876


 20%|██        | 1/5 [16:43<1:06:54, 1003.70s/it]

loss=0.004277772562201793, epoch=1


 40%|████      | 2/5 [34:17<51:39, 1033.08s/it]  

...

Epoch 2, NDCG@10: 0.1854647188036368
Epoch 2, HR@10: 0.2939453125
Epoch 2, Cov@10: 0.02698118162679261
loss=0.0042547431139116606, epoch=3


 80%|████████  | 4/5 [1:09:12<17:25, 1045.22s/it]

Epoch 3, NDCG@10: 0.18876920816031142
Epoch 3, HR@10: 0.29296875
Epoch 3, Cov@10: 0.02802736435143129
loss=0.0042655951795531975, epoch=4
Epoch 4, NDCG@10: 0.19076465098257944
Epoch 4, HR@10: 0.3056640625
Epoch 4, Cov@10: 0.026959655233281525


100%|██████████| 5/5 [1:25:58<00:00, 1031.69s/it]



Peak training memory, mb: 12089.61
Training epoches: 5
Training time, m: 85.97


In [15]:
from eval_utils import model_evaluate, sasrec_model_scoring, get_test_scores

model_e.eval()
val_scores = sasrec_model_scoring(model_e, testset_valid_temp_cut, data_description_temp, device)
val_recs = topn_recommendations(val_scores, topn=10)
val_metrics = model_evaluate(val_recs, holdout_valid_temp_cut, data_description_temp)
ndcg_ = val_metrics['ndcg@10']
print(ndcg_)

0.029985753708965737


In [9]:
torch.save(model_e.state_dict(), 'saved_models/model_e0_nonsvd.pt')

In [8]:
base_config_bce = dict(
    manual_seed = 123,
    sampler_seed = 123,
    init_emb_svd = None,
    lin_layer_dim = 128,
    num_epochs = 1, #3 10 22 100&dropout0.9&hd32&bs1000
    maxlen = 100,
    hidden_units = 128,
    dropout_rate = 0.3,
    num_blocks = 2,
    num_heads = 2,
    batch_size = 128,
    learning_rate = 1e-3,
    fwd_type = 'bce',
    l2_emb = 0,
    patience = 10,
    skip_epochs = 1,
    n_neg_samples=600,
    sampling='without_rep'
)

model_e = build_sasrec_model(base_config_bce,
                             training_temp,
                             data_description_temp,
                             testset_valid_temp_cut,
                             holdout_valid_temp_cut,
                             device,
                             task=None,
                             log=None,
                             item_emb_svd=item_embs)

  0%|          | 0/1 [00:00<?, ?it/s]

loss=0.008304253079943452, epoch=0
Epoch 0, NDCG@10: 0.07723376882934793
Epoch 0, HR@10: 0.1162109375
Epoch 0, Cov@10: 0.011671610561709712


100%|██████████| 1/1 [16:42<00:00, 1002.67s/it]



Peak training memory, mb: 12440.19
Training epoches: 1
Training time, m: 16.71


In [9]:
torch.save(model_e.state_dict(), 'saved_models/model_e1.pt')

In [10]:
base_config_bce = dict(
    manual_seed = 123,
    sampler_seed = 123,
    init_emb_svd = None,
    lin_layer_dim = 128,
    num_epochs = 5, #3 10 22 100&dropout0.9&hd32&bs1000
    maxlen = 100,
    hidden_units = 128,
    dropout_rate = 0.9,
    num_blocks = 2,
    num_heads = 2,
    batch_size = 128,
    learning_rate = 1e-3,
    fwd_type = 'bce',
    l2_emb = 0,
    patience = 10,
    skip_epochs = 1,
    n_neg_samples=600,
    sampling='without_rep'
)

model_e = build_sasrec_model(base_config_bce,
                             training_temp,
                             data_description_temp,
                             testset_valid_temp_cut,
                             holdout_valid_temp_cut,
                             device,
                             task=None,
                             log=None,
                             item_emb_svd=item_embs)

  0%|          | 0/5 [00:00<?, ?it/s]

loss=0.011270346139770359, epoch=0
Epoch 0, NDCG@10: 0.042245738184161705
Epoch 0, HR@10: 0.0556640625
Epoch 0, Cov@10: 0.002398040237134751


 20%|██        | 1/5 [16:39<1:06:37, 999.45s/it]

loss=0.00786146905593081, epoch=1
Epoch 1, NDCG@10: 0.0490509828921286
Epoch 1, HR@10: 0.0654296875
Epoch 1, Cov@10: 0.003470054633986731


 40%|████      | 2/5 [33:05<49:34, 991.38s/it]  

...

 80%|████████  | 4/5 [1:06:02<16:29, 989.63s/it]

Epoch 3, NDCG@10: 0.05045065840100071
Epoch 3, HR@10: 0.0673828125
Epoch 3, Cov@10: 0.004507626801220977
loss=0.007715946873599973, epoch=4


100%|██████████| 5/5 [1:22:34<00:00, 990.96s/it]

Epoch 4, NDCG@10: 0.051895019067258796
Epoch 4, HR@10: 0.0712890625
Epoch 4, Cov@10: 0.004417215948474424



Peak training memory, mb: 12440.19
Training epoches: 5
Training time, m: 82.58


In [11]:
torch.save(model_e.state_dict(), 'saved_models/model_e2.pt')

In [15]:
from eval_utils import get_test_scores
from train import prepare_sasrec_model

testset = pd.read_csv('testset.csv')
# item_embs = torch.load("./saved_models/item_embs.pt", map_location=torch.device(device))
item_embs = None

base_config_bce = dict(
    manual_seed = 123,
    sampler_seed = 123,
    init_emb_svd = None,
    lin_layer_dim = -1,
    num_epochs = 5, #3 10 22 100&dropout0.9&hd32&bs1000
    maxlen = 100,
    hidden_units = 128,
    dropout_rate = 0.3,
    num_blocks = 2,
    num_heads = 2,
    batch_size = 128,
    learning_rate = 1e-3,
    fwd_type = 'bce',
    l2_emb = 0,
    patience = 10,
    skip_epochs = 1,
    n_neg_samples=600,
    sampling='without_rep'
)

# model, _, _, _ = prepare_sasrec_model(base_config_bce, training_temp, data_description_temp, device, item_embs)
# model.load_state_dict(torch.load('../sasrec_cql/RECE/saved_models/model_e2_nonsvd.pt', map_location=torch.device(device)))
# model.eval()

model, _, _, _ = prepare_sasrec_model(base_config_bce, training_temp, data_description_temp, device, item_embs)
model.load_state_dict(torch.load('../sasrec_cql/RECE/saved_models/model_e2_nonsvd.pt', map_location=torch.device(device)))
model.eval()

testset_, holdout_ = leave_one_out(
    testset, target='timestamp', sample_top=True, random_state=0
)

test_size = 5000
test_users = np.intersect1d(holdout_['userid'].unique(), testset_['userid'].unique())
if test_size < len(test_users):
    test_users  = np.random.choice(test_users, size=test_size, replace=False)
testset = testset_[testset_['userid'].isin(test_users)].sort_values(by=['userid', 'timestamp'])
holdout = holdout_[holdout_['userid'].isin(test_users)].sort_values(['userid'])

get_test_scores(model, data_description_temp, testset, holdout, device)

{'hr@1': 0.082,
 'mrr@1': 0.082,
 'ndcg@1': 0.082,
 'cov@1': 0.010810554821266355,
 'hr@5': 0.1444,
 'mrr@5': 0.10323,
 'ndcg@5': 0.11337880510435107,
 'cov@5': 0.03384810115682839,
 'hr@10': 0.182,
 'mrr@10': 0.10829690476190476,
 'ndcg@10': 0.1255870218388335,
 'cov@10': 0.05366099374443004,
 'hr@20': 0.2452,
 'mrr@20': 0.11258995352428787,
 'ndcg@20': 0.1414485607144471,
 'cov@20': 0.08207152790035863,
 'hr@50': 0.33,
 'mrr@50': 0.11529694376092786,
 'ndcg@50': 0.15828277602013582,
 'cov@50': 0.13432039023046158}

In [11]:
# то, что ниже, тебе сейчас не нужно особо

### Check offline performance

In [604]:
test_sequences = data_to_sequences(testset, data_description_full)
test_sequences

userid
1       [2935, 1160, 1552, 941, 2117, 1633, 3136, 2566...
2       [1090, 1102, 1109, 2479, 1183, 2702, 1117, 108...
3       [573, 2618, 3260, 1762, 1307, 1755, 1156, 1259...
4       [1102, 1008, 3194, 463, 3253, 253, 1088, 1090,...
5       [2479, 832, 843, 1140, 346, 2618, 1033, 1981, ...
                              ...                        
6001    [3464, 3207, 1562, 1544, 2635, 1139, 1242, 889...
6002    [1736, 440, 1740, 2049, 2849, 265, 961, 2898, ...
6016    [3570, 3463, 3507, 1344, 2880, 3601, 27, 2466,...
6028                                               [2755]
6040    [2932, 2348, 1104, 3092, 3148, 1148, 1160, 186...
Name: itemid, Length: 1783, dtype: object

In [602]:
def recommend_sequential(
    model,
    target_seq: Union[list, np.ndarray],
    seen_seq: Union[list, np.ndarray],
    topn: int,
    *,
    user: Optional[int] = None
):
    '''Given an item sequence and a sequence of next target items,
    predict top-n candidates for each next step in the target sequence.
    '''
    model.eval()
    predictions = predict_sequential(model, target_seq[:-1], seen_seq, user=user)
    predictions[:, seen_seq] = -np.inf
    for k in range(1, predictions.shape[0]):
        predictions[k, target_seq[:k]] = -np.inf
    predicted_items = np.apply_along_axis(topidx, 1, predictions, topn)
    return predicted_items

def predict_sequential(model, target_seq, seen_seq, user):

    maxlen = base_config1['maxlen']

    n_seen = len(seen_seq)
    n_targets = len(target_seq)
    seq = np.concatenate([seen_seq, target_seq])

    with torch.no_grad():
        pad_seq = torch.as_tensor(
            np.pad(
                seq, (max(0, maxlen-n_seen), 0),
                mode = 'constant',
                constant_values = model.pad_token
            ),
            dtype = torch.int64,
            device = device
        )
        log_seqs = torch.as_strided(pad_seq[-n_targets-maxlen:], (n_targets+1, maxlen), (1, 1))
        log_feats = model.log2feats(log_seqs)[:, -1, :]
        item_embs = model.item_emb.weight
        logits = item_embs.matmul(log_feats.unsqueeze(-1)).squeeze(-1)


    # predictions = item_embs.matmul(final_feat.unsqueeze(-1)).squeeze(-1)
    
    return logits.detach().cpu().numpy()

def topidx(arr, topn):
    parted = np.argpartition(arr, -topn)[-topn:]
    return parted[np.argsort(-arr[parted])]

In [603]:
models = {
    'SASRec 1': model_D_sasrec,
    'SASRec 2': model_e,
    'SASRec 3': model_e3,
    'SASRec 4': model_e5
}

topn = 10
test_data = test_sequences
results_list = []


# Loop over each model
for model_name, model in models.items():
    cum_hits = 0
    cum_reciprocal_ranks = 0.
    cum_discounts = 0.
    unique_recommendations = set()
    total_count = 0
    cov = []

    # Loop over each user and test sequence
    for user, test_seq in tqdm(test_data.items()):
        seen_seq = test_seq[:1]
        test_seq = test_seq[1:]
        num_predictions = len(test_seq)
        if not num_predictions:  # if no test items left - skip user
            continue

        # Get predicted items
        predicted_items = recommend_sequential(model, test_seq, seen_seq, topn, user=user)
        
        # Compute hit steps and indices
        hit_steps, hit_index = np.where(predicted_items == np.atleast_2d(test_seq).T)
        cov.append(len(np.unique(predicted_items.ravel()))/ data_description_temp['n_items'])

        num_hits = hit_index.size
        if num_hits:
            cum_hits += num_hits
            cum_reciprocal_ranks += np.sum(1. / (hit_index + 1))
            cum_discounts += np.sum(1. / np.log2(hit_index + 2))
        total_count += num_predictions

    # Compute evaluation metrics for the current model
    hr = cum_hits / total_count
    mrr = cum_reciprocal_ranks / total_count
    dcg = cum_discounts / total_count
    cov = np.mean(cov)

    # Append the results for this model to the results list
    results_list.append(pd.DataFrame(
        data={'score': [hr, mrr, dcg, cov]},
        index=[f'{metric}@{topn}' for metric in ['HR', 'MRR', 'NDCG', 'COV']],
        # columns=[model_name]  # Label results by model name
    ))
    
final_results = pd.concat(results_list, axis=1)
final_results.columns = models.keys()
final_results

1783it [00:19, 89.22it/s] 
1783it [00:19, 89.89it/s] 
1783it [00:19, 89.48it/s] 
1783it [00:19, 90.40it/s] 


,SASRec 1,SASRec 2,SASRec 3,SASRec 4
HR@10,0.239497,0.221020,0.179104,0.028692
MRR@10,0.094344,0.085006,0.067651,0.009201
NDCG@10,0.128065,0.116584,0.093494,0.013647
COV@10,0.067123,0.065069,0.060589,0.005394


In [222]:
model = model_D_sasrec
topn = 10
##################################
test_data = test_sequences

cum_hits = 0
cum_reciprocal_ranks = 0.
cum_discounts = 0.
unique_recommendations = set()
total_count = 0
for user, test_seq in tqdm(test_data.items()):
    
    seen_seq = test_seq[:1]
    test_seq = test_seq[1:]
    num_predictions = len(test_seq)
    if not num_predictions: # if no test items left - skip user
        continue
    predicted_items = recommend_sequential(model, test_seq, seen_seq, topn, user=user)
    hit_steps, hit_index = np.where(predicted_items == np.atleast_2d(test_seq).T)
    unique_recommendations.update(predicted_items.ravel())

    num_hits = hit_index.size
    if num_hits:
        cum_hits += num_hits
        cum_reciprocal_ranks += np.sum(1. / (hit_index+1))
        cum_discounts += np.sum(1. / np.log2(hit_index+2))
    total_count += num_predictions

hr = cum_hits / total_count
mrr = cum_reciprocal_ranks / total_count
dcg = cum_discounts / total_count
cov = len(unique_recommendations) / data_description_temp['n_items']
resultsD = pd.DataFrame(
    data = {'score': [hr, mrr, dcg, cov]},
    index = [f'{metric}@{topn}' for metric in ['HR', 'MRR', 'NDCG', 'COV']]
)
resultsD

1783it [00:20, 87.20it/s] 


,score
HR@10,0.239497
MRR@10,0.094344
NDCG@10,0.128065
COV@10,0.795790
